### Sample Code To Cover

In [18]:

public interface IEventBusProvider
{
  void Send(object message);
}

class Service
{
  private readonly IEventBusProvider _eventBusProvider;
  public Service(IEventBusProvider eventBusProvider)
  {
    _eventBusProvider = eventBusProvider;
  }

  public void FireAndForget()
  {
    // Do Smth
    _eventBusProvider.Send(new { Message = "Hello" });
  }
}



### Using Moq

In [19]:
#r "nuget:Moq, 4.18.4"

using Moq;

// Arrange
var eventBusMock = new Mock<IEventBusProvider>();
var service = new Service(eventBusMock.Object);

// Act
service.FireAndForget();

// Assert
eventBusMock.Verify(x => x.Send(It.IsAny<object>()), Times.Once);

Installed Packages Moq, 4.18.4


warning CS1701: Assuming assembly reference 'System.Linq.Expressions, Version=6.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'Moq' matches identity 'System.Linq.Expressions, Version=7.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Linq.Expressions', you may need to supply runtime policy



### Goal Syntax

In [ ]:
var eventBusMock = new MockObject<IEventBusProvider>();
var service = new Service(eventBusMock.Object);
service.FireAndForget();
eventBusMock.Verify(x => x.Send(Arg.IsAny<object>()), Executed.Once);

### Symple Implementation

#### The most Simple: class to verify execution count

In [21]:
public sealed class Executed
{
  private readonly Func<int, bool> _validate;
  private readonly string _message;

  public Executed(Func<int, bool> validate, string message)
  {
    _validate = validate;
    _message = message;
  }

  public void Verify(int count)
  {
    if (!_validate.Invoke(count))
    {
      throw new Exception($"Executed wrong number of times. Expected: {_message} Executed: {count}");
    }
  }

  public static Executed Once => new (x => x == 1, "Once");
  public static Executed Never => new (x => x == 0, "Never");
  public static Executed AtLeast(int times) => new (x => x >= times, $"At least {times}");
  public static Executed AtMost(int times) => new (x => x <= times, $"At most {times}");
}

public class Arg
{
  public static T IsAny<T>() => throw new InvalidOperationException();
}

#### Factory for Proxies

In [23]:
using Castle.DynamicProxy;

public class ProxyFactory
{
  private readonly ProxyGenerator _generator = new ();

  public T GetProxy<T>(IInterceptor interceptor) where T:class => _generator.CreateInterfaceProxyWithoutTarget<T>(interceptor);
}

public static class FactoryProvider 
{
  public static ProxyFactory Factory {get;} = new ProxyFactory();
}

#### Simple MockObject (v1)

In [24]:
using System.Linq.Expressions;
using IInvocation = Castle.DynamicProxy.IInvocation; // It is conflict with Moq.Invocation which in fact wraps Castle.Core

public class MockObject<T>  where T : class 
{
  private readonly Lazy<T> _lazy;
  private readonly MockInterceptor _interceptor = new();
  
  public MockObject() 
  {
    _lazy = new Lazy<T>(CreateInstance);
  }

  public T Object => _lazy.Value;

  private T CreateInstance() => FactoryProvider.Factory.GetProxy<T>(_interceptor);
}

public class MockInterceptor : StandardInterceptor
{
  protected override void PreProceed(IInvocation invocation) => Console.WriteLine("PreProceed");

  protected override void PerformProceed(IInvocation invocation) => Console.WriteLine("PerformProceed");
}

#### Sample Syntax Check (Is is at least compiles?)

In [25]:
var eventBusMock = new MockObject<IEventBusProvider>();
var service = new Service(eventBusMock.Object);
service.FireAndForget();
eventBusMock.Verify(x => x.Send(Arg.IsAny<object>()), Executed.Once);


(4,14): error CS1061: 'MockObject<IEventBusProvider>' does not contain a definition for 'Verify' and no accessible extension method 'Verify' accepting a first argument of type 'MockObject<IEventBusProvider>' could be found (are you missing a using directive or an assembly reference?)



Error: compilation error

There is an error. Verify Method should be added to MockObject.
Let's guess it's signature and implement it later

```CSharp
// Mock<T> Verify:
public void Verify(Expression<Action<T>> expression, Times times);

// So our vatiant would be simmilar
public void Verify(Expression<Action<T>> expression, Executed times) {}

#### How verify can be implemented?

In fact really easy.
We have this MockInterceptor class and we should store Invocations there.
Then when verified is called we should *Filter* by the target parameter values and then count invocations

```CSharp
public class MockInterceptor : StandardInterceptor
{
  protected override void PreProceed(IInvocation invocation) => Console.WriteLine("PreProceed");

  protected override void PerformProceed(IInvocation invocation) => Console.WriteLine("PerformProceed");
}
```

#### Interceptor (V2) with collecting invocations

In [26]:
public class MockInvocationMatch
{
  private readonly Expression _expression;
  public MockInvocationMatch(Expression expression)
  {
    _expression = expression;
  }
  public bool IsMatch(MockInvocation invocation) => true;
}

public class MockInvocation
{
  private readonly IInvocation _invocation;
  public MockInvocation(IInvocation invocation)
  {
    _invocation = invocation;
  }
}

In [27]:
public class MockInterceptor : StandardInterceptor
{
  private readonly List<MockInvocation> _invocations = new();

  public IReadOnlyList<MockInvocation> Invocations => _invocations;

  protected override void PreProceed(IInvocation invocation)
  {
    _invocations.Add(new MockInvocation(invocation));
    base.PreProceed(invocation);
  }

  protected override void PerformProceed(IInvocation invocation) {}
}

#### And now we can use Invocations List in MockObject.Verify
```CSharp
  public void Verify(Expression<Action<T>> action, Executed times)
  {
    var invocationMatch = new MockInvocationMatch(action);
    times.Verify(_interceptor.Invocations.Count(x => x.IsMatch(invocationMatch)));
  }
```

In [28]:
public class MockInterceptor : StandardInterceptor
{
  private readonly List<MockInvocation> _invocations = new();

  public IReadOnlyList<MockInvocation> Invocations => _invocations;

  protected override void PreProceed(IInvocation invocation)
  {
    _invocations.Add(new MockInvocation(invocation));
    base.PreProceed(invocation);
  }

  protected override void PerformProceed(IInvocation invocation)
  {
  }
}

public class MockObject<T>  where T : class 
{
  private readonly Lazy<T> _lazy;
  private readonly MockInterceptor _interceptor = new();
  
  public MockObject() 
  {
    _lazy = new Lazy<T>(CreateInstance);
  }

  public void Verify(Expression<Action<T>> action, Executed times)
  {
    var invocationMatch = new MockInvocationMatch(action);
    times.Verify(_interceptor.Invocations.Count(x => invocationMatch.IsMatch(x)));
  }

  public T Object => _lazy.Value;

  private T CreateInstance() => FactoryProvider.Factory.GetProxy<T>(_interceptor);
}

#### Let's verify out goal Syntax again

In [31]:
var eventBusMock = new MockObject<IEventBusProvider>();
var service = new Service(eventBusMock.Object);
service.FireAndForget();
eventBusMock.Verify(x => x.Send(Arg.IsAny<object>()), Executed.Once);

In [30]:
eventBusMock.Verify(x => x.Send(Arg.IsAny<object>()), Executed.AtLeast(4));

Error: System.Exception: Executed wrong number of times. Expected: At least 4 Executed: 1
   at Submission#23.Executed.Verify(Int32 count)
   at Submission#30.MockObject`1.Verify(Expression`1 action, Executed times)
   at Submission#32.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

####  And Important Note
Despite the Arg.IsAny<T> *always* throws exception out code executes OK.
```CSharp
public class Arg
{
  public static T IsAny<T>() => throw new InvalidOperationException();
}
```

It's because it is never executed and Passed to verify as expression.
```CSharp 
// So our vatiant would be simmilar
public void Verify(Expression<Action<T>> expression, Executed times) {}
```